<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW1/blob/main/BigData_HW1_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

# Download File

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id='1-OCBGBtKoY_PadKHcXDyWxHQ2BS8Nulo'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('hw1.zip')

!unzip hw1.zip

Archive:  hw1.zip
   creating: hw1-files/
   creating: hw1-files/q3/
  inflating: hw1-files/q3/patches.csv  
  inflating: hw1-files/q3/lsh.py     
   creating: hw1-files/q1/
  inflating: hw1-files/q1/dataset1.txt  
   creating: hw1-files/q1/.ipynb_checkpoints/
   creating: hw1-files/q2/
  inflating: hw1-files/q2/games_library.txt  
   creating: hw1-files/.ipynb_checkpoints/


# PySpark Setting

In [3]:
!pip install pyspark
!pip install -q findspark

In [4]:
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.7/dist-packages/pyspark'

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [6]:
spark

# Load Dataset

In [15]:
#df = spark.read.format("csv").option("sep","\t").option("header", False).load("/content/hw1-files/q1/dataset1.txt")

In [86]:
def convert_string_to_list(string):
  if string=="":
    return []
  else:
    return [int(item) for item in string.split(",")]

In [87]:
import pandas as pd
df = pd.read_csv("/content/hw1-files/q1/dataset1.txt", sep="\t",names=["id", "friend_id"])

In [88]:
df = df.fillna("")
df["friend_id"] = df["friend_id"].apply(convert_string_to_list)

In [94]:
df.to_dict('records')[:3]

[{'friend_id': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94],
  'id': 0},
 {'friend_id': [0,
   5,
   20,
   135,
   2409,
   8715,
   8932,
   10623,
   12347,
   12846,
   13840,
   13845,
   14005,
   20075,
   21556,
   22939,
   23520,
   28193,
   29724,
   29791,
   29826,
   30691,
   31232,
   31435,
   32317,
   32489,
   34394,
   35589,
   35605,
   35606,
   35613,
   35633,
   35648

In [142]:
rdd=spark.sparkContext.parallelize(df.to_dict('records'), 2)

In [143]:
def map_function(item):
  fids = item["friend_id"]

  output = []
  for fid1 in fids:
    for fid2 in fids:
      key = (fid1, fid2)
      value = 1
      output.append((key, value))
  return output

In [144]:
def reduce_function(value1, value2):
  return value1 + value2  

In [145]:
mapped_rdd = rdd.flatMap(map_function)

In [146]:
reduced_rdd = mapped_rdd.reduceByKey(reduce_function)

In [147]:
reduced_rdd

PythonRDD[48] at RDD at PythonRDD.scala:53

In [149]:
reduced_rdd.take(5)

[((54, 40), 1),
 ((85, 81), 1),
 ((31435, 22939), 1),
 ((629, 29791), 1),
 ((9847, 21), 1)]

In [151]:
for item in reduced_rdd.collect():
  if item[1]>10:
    print(item)

Streaming output truncated to the last 5000 lines.
((14130, 13993), 16)
((23592, 23617), 19)
((34258, 22357), 16)
((4281, 4246), 13)
((14175, 14026), 11)
((34571, 34200), 15)
((23717, 23814), 14)
((15314, 627), 12)
((34688, 22355), 16)
((34602, 34207), 14)
((13917, 14244), 14)
((37506, 41851), 11)
((44102, 15383), 23)
((37362, 37009), 13)
((4246, 15843), 34)
((10066, 37017), 14)
((37135, 36988), 34)
((13939, 13966), 19)
((10058, 37291), 17)
((36916, 37073), 19)
((10000, 44053), 17)
((36667, 44050), 11)
((23279, 39268), 14)
((36944, 36877), 18)
((36728, 36707), 13)
((49221, 49196), 19)
((16878, 35633), 12)
((24698, 33559), 12)
((23476, 39479), 11)
((23404, 23377), 18)
((42255, 42260), 15)
((12578, 14207), 15)
((4612, 4555), 15)
((626, 11405), 12)
((23657, 13982), 13)
((21275, 14242), 11)
((34426, 34159), 14)
((2190, 2189), 14)
((18005, 18024), 11)
((19357, 34288), 11)
((23608, 13965), 19)
((13889, 13956), 12)
((13956, 13993), 12)
((23592, 23695), 20)
((14207, 14194), 11)
((34304, 34219)